**Insurance companies can't predict the future, so how do they come up with such quote?**

By understanding the relationship between the demographics of primary beneficiaries (age, sex, BMI, number of children, smoker and location) and their medical costs, insurance companies will then (i) compare you to their data, (ii) predict your medical costs, and (iii) calculate a corresponding insurance quote. 

Let's take a look at the first two steps (grouping you and predicting your medical costs) with the following example:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline
params={"figure.facecolor":(0.0,0.0,0.0,0),
        "axes.facecolor":(1.0,1.0,1.0,1.0),
        "savefig.facecolor":(0.0,0.0,0.0,0)}
plt.rcParams.update(params)

In [ ]:
df=pd.read_csv("../input/insurance/insurance.csv")
df.columns=df.columns.to_series().apply(lambda x:x.title() if x!="bmi" else x.upper())
df.head()

In [ ]:
df.shape

The dataset has 1338 observations/individuals which can be described in the following seven features:

* *Age*: the age of primary beneficiary

* *Sex*: female or male

* *BMI*: Body Mass Index (kg/m^2), a BMI between 18.5 kg/m^2 to 24.9 kg/m^2 indicates a healthy individual while anything less indicates underweight and more indicates overweight

* *Children*: the number of children/dependents covered by insurance

* *Smoker*: if the primary beneficiary smokes

* *Region*: the beneficiary's residential area in the US (northeast, southeast, southwest, northwest)

* *Charges*: medical costs billed by the insurance

In [ ]:
df.info()

There are no null values, but there is a mixture of categorical and numerical features (the categorical features will later be changed into numerical format).

Let's further examine each feature:

#### *Age*

In [ ]:
sns.scatterplot(x=df["Age"],y=df["Charges"],hue=df["Smoker"],palette={"yes":"salmon","no":"skyblue"})
plt.legend(bbox_to_anchor=(1,0.5),loc=6)

* There is a slight positive correlation between the age and charges.
* The data can also be separated into three groupings (low, average and high charges) depending on if the individual was a smoker.

#### *Sex*

In [ ]:
sns.boxplot(x=df["Sex"],y=df["Charges"],palette={"female":"lemonchiffon","male":"lightgreen"})

* Including the "outliers", both females and males have similar range.
* Excluding the "outliers", males have higher charges.

#### *BMI*

In [ ]:
sns.scatterplot(x=df["BMI"],y=df["Charges"],hue=df["Smoker"],palette={"yes":"salmon","no":"skyblue"})
plt.legend(bbox_to_anchor=(1,0.5),loc=6)

* Charges do not necessarily increase with BMI, it depends on if the individual smokes.

#### Number of *Children*

In [ ]:
sns.boxplot(x=df["Children"],y=df["Charges"],palette="Set2")

* Having more children may be more expensive than those with no or less children.

#### *Smoker* Status

In [ ]:
sns.boxplot(x=df["Smoker"],y=df["Charges"],palette={"yes":"salmon","no":"skyblue"})

* Smokers have a chargers than non-smokers.

#### *Region*

In [ ]:
sns.boxplot(x=df["Region"],y=df["Charges"],palette="Set3")

* On average there is no obvious relationship between the region and charges, but those in the southeast have a larger spread of higher charges.

Let us now change the categorical features into numerical format:

In [ ]:
df["Sex"]=pd.get_dummies(df["Sex"],drop_first=True)
#where 0=female and 1=male

In [ ]:
df["Smoker"]=pd.get_dummies(df["Smoker"],drop_first=True)
#where 0=non-smoker and 1=smoker

In [ ]:
df_regions=pd.get_dummies(df["Region"])
df=df.join(df_regions)
df.drop(["Region"],axis=1,inplace=True)

The dataset now looks like this:

In [ ]:
df.head()

We shall continue exploring our target variable, *Charges*:

In [ ]:
sns.distplot(a=df["Charges"],color="plum",hist_kws=dict(edgecolor="black",linewidth=2),bins=20,kde=False)

In [ ]:
print("Charges have a skewness of {:.2f}.".format(stats.skew(df["Charges"])))

In [ ]:
df["Charges"].describe()

In [ ]:
plt.figure(figsize=(12,4))
sns.boxplot(df["Charges"],color="plum")

In [ ]:
mask=np.zeros_like(df.corr(),dtype=np.bool)
mask[np.triu_indices_from(mask)]=True
sns.heatmap(df.corr(),annot=False,mask=mask,square=True,cmap="PuRd",linewidths=1,linecolor="white")

In [ ]:
df.corr()

In [ ]:
a=df["Charges"].loc[df["Smoker"]==1]
b=df["Charges"].loc[df["Smoker"]==0]

stats.ttest_ind(a,b)

* The charges can range from USD 1121 to USD 63770 with an average charge of USD 13270 and a right skewness of 1.51.
* Charges are strongly correlated (0.79) and statistically significant (8e-283 pvalue) to whether the individual smokes or not.
* Age and BMI are weakly correlated to charges; sex, number of children and region have almost no correlation with charges.

The data will be split into a training and a testing set to train the model:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x=df.drop(["Charges"],axis=1)
y=df["Charges"]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=7)

In [ ]:
print("Training set x shape:",x_train.shape,"and y shape",y_train.shape)
print("Testing set x shape:",x_test.shape,"and y shape",y_test.shape)

As the data includes a range of continuous and discrete variables, the data will need to be scaled to ensure those larger values to not give an inaccurate higher weighting. The training set will be fit to the scaler and transformed, while the testing set will only be transformed to avoid any data leakage:

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler=StandardScaler()

In [ ]:
x_train=scaler.fit_transform(x_train)

In [ ]:
x_test=scaler.transform(x_test)

There are nine independent variables that can predict the target variable, *Charges*, but not all nine will have the same predictive power. Based on their scores (larger is better) and pvalues (smaller is better) it can then be decided if all nine or a selection of the variables will be used in the model. If only a selection will be used - as with scaling - it is important to fit and transform the training set while only transforming the testing set to prevent any data leakage:

In [ ]:
from sklearn.feature_selection import SelectKBest,f_regression

In [ ]:
selector=SelectKBest(f_regression,k="all")

In [ ]:
x_selection=selector.fit(x_train,y_train)

In [ ]:
x.columns

In [ ]:
x_selection.scores_

In [ ]:
x_selection.pvalues_

* As discovered previously, smoking is a strong predictor with the largest score (1e+03) and smallest pvalue (8e-189). This is followed by age and BMI.
* All of the nine variables are statistically significant with a pvalue of less than 0.05, so all the nine variables will be used in the model. Hence no need to transform the data.

Various regression models will be tested and evaluated based on their cross validated accuracy score:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
names=["LinearRegression","kNN","SVM","RandomForest"]
models=[LinearRegression(),KNeighborsRegressor(),SVR(),RandomForestRegressor()]
meancross=[]
stdcross=[]

In [ ]:
for model in models:
    model=model
    model.fit(x_train,y_train)
    rcross=cross_val_score(model,x_train,y_train,cv=5)
    meancross.append(rcross.mean())
    stdcross.append(rcross.std())

In [ ]:
results=pd.DataFrame({"Name":names,"Model":models,"RCrossMean":meancross,"RCrossStd":stdcross})
results.sort_values("RCrossMean",ascending=False)

Although the RandomForest model has the second largest spread of cross validation scores, the RandomForest performed the best so let's explore more ensemble models:

In [ ]:
from sklearn.ensemble import BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor 

In [ ]:
names=["RandomForest","Bagging","GradientBoost","AdaBoost"]
models=[RandomForestRegressor(),BaggingRegressor(),GradientBoostingRegressor(),AdaBoostRegressor()]
meancross=[]
stdcross=[]

In [ ]:
for model in models:
    model=model
    model.fit(x_train,y_train)
    rcross=cross_val_score(model,x_train,y_train,cv=5)
    meancross.append(rcross.mean())
    stdcross.append(rcross.std())

In [ ]:
results=pd.DataFrame({"Name":names,"Model":models,"RCrossMean":meancross,"RCrossStd":stdcross})
results.sort_values("RCrossMean",ascending=False)

Out of the ensemble models, GradientBoost performed the best even though - again - it has the second largest spread of cross validation scores.

In [ ]:
model=GradientBoostingRegressor()
model.fit(x_train,y_train)
y_predict=model.predict(x_test)

In [ ]:
sns.distplot(y_test,color="yellowgreen",label="Actual values")
sns.distplot(y_predict,color="lightcoral",label="Predicted values")
plt.legend()
plt.xlim([0,60000])

In [ ]:
f,axes=plt.subplots(2,1,sharex=True)
sns.boxplot(y_test,color="yellowgreen",whis=5,ax=axes[0])
sns.boxplot(y_predict,color="lightcoral",whis=5,ax=axes[1])
axes[0].set_xlabel("")
plt.xlabel("Charges")
axes[0].set_ylabel("Actual Price")
axes[1].set_ylabel("Predicted Price")

In [ ]:
print("The model's score is {:.2f}.".format(model.score(x_test,y_test)))

In [ ]:
a=y_test
b=y_predict

stats.ttest_ind(a,b)

In [ ]:
results=pd.DataFrame({"Actual":y_test,"Predicted":y_predict})

smin=[stats.tmin(results[i]) for i in results.columns]
smax=[stats.tmax(results[i]) for i in results.columns]
smean=[stats.tmean(results[i]) for i in results.columns]
sstd=[stats.tstd(results[i]) for i in results.columns]
sskew=[stats.skew(results[i]) for i in results.columns]
skurtosis=[stats.kurtosis(results[i]) for i in results.columns]

stable=pd.DataFrame({"Min":smin,"Max":smax,"Mean":smean,"Standard Deviation":sstd,"Skew":sskew,"Kurtosis":skurtosis},index=results.columns)
stable

The base model gave an accuracy score of almost 86% and can be improved by obtaining more data, conducting further feature selection (as we utilised all the features) and perfoming parameter tuning (we have only used the default parameters). Nevertheless the actual and predicted resulare not statistically different as while the minimum value is rather off, the maximum value, mean, standard deviation, skew and kurtosis of the actual and predicted values are still similar.

Try to predict your medical costs by inputting the relevant values in place for the '0's below:

In [ ]:
age=[0] # your age as a whole number
sex=[0] # 0 if you are a female OR 1 if you are a male
bmi=[0] # your calculated BMI as a whole or decimal number
children=[0] # your number of children/dependents as a whole number
smoker=[0] # 0 if you do not smoke OR 1 if you do smoke
northeast=[0] # 0 if you live in the northwest/southeast/southwest OR 1 if you live in the northeast
northwest=[0] # 0 if you live in the northeast/southeast/southwest OR 1 if you live in the northwest
southeast=[0] # 0 if you live in the northeast/northwest/southwest OR 1 if you live in the southeast
southwest=[0] # 0 if you live in the northeast/northwest/southeast OR 1 if you live in the southwest

In [ ]:
if (age==[0])&(sex==[0])&(bmi==[0])&(children==[0])&(smoker==[0])&(northeast==[0])&(northwest==[0])&(southeast==[0])&(southwest==[0]):
    print("Your predicted medical costs is USD x.")
else:
    your_data=pd.DataFrame({"Age":age,"Sex":sex,"BMI":bmi,"Children":children,"Smoker":smoker,"northeast":northeast,"northwest":northwest,"southeast":southeast,"southwest":southwest})
    your_data=scaler.transform(your_data)
    your_costs=model.predict(your_data)
    print("Your predicted medical costs is USD {:.2f}.".format(your_costs[0]))

Does this value seem right to you?